In [ ]:
from bs4 import BeautifulSoup  # 对html格式的文件, 处理
import requests
import pandas as pd
from time import sleep

In [ ]:
def get_info(res, get_content=True):
    '''
    获取指定的内容(题目和评论)或问题id
    '''
    contents_dict = res.json()["data"]
    if get_content:
        contents_list = []
        for item in contents_dict:
            content = ""
            title_name = item["question"]["title"]
            # voteup_count = item["voteup_count"]
            html = item["content"]
            bf = BeautifulSoup(html)
            for p in bf.stripped_strings:
                content += p
            contents_list.append((title_name, content))
        return contents_list
    else:
        question_id_list = []
        for item in contents_dict:
            content = ""
            question_id = str(item["id"])
            question_id_list.append(question_id)
        return question_id_list

In [ ]:
def grap_web(question_id):
    '''
    用于获取该问题号对应的所有评论
    '''
    # 爬取网站, num控制每次爬取评论数目
    num = 5
    offset = 0 # 起始值
    i = 0
    ## 准备爬取的网站
    base_url = "https://www.zhihu.com/api/v4/questions/" + question_id + "/answers?include=data%5B*%5D.is_normal%2Cadmin_closed_comment%2Creward_info%2Cis_collapsed%2Cannotation_action%2Cannotation_detail%2Ccollapse_reason%2Cis_sticky%2Ccollapsed_by%2Csuggest_edit%2Ccomment_count%2Ccan_comment%2Ccontent%2Ceditable_content%2Cvoteup_count%2Creshipment_settings%2Ccomment_permission%2Ccreated_time%2Cupdated_time%2Creview_info%2Crelevant_info%2Cquestion%2Cexcerpt%2Crelationship.is_authorized%2Cis_author%2Cvoting%2Cis_thanked%2Cis_nothelp%2Cis_labeled%2Cis_recognized%2Cpaid_info%2Cpaid_info_content%3Bdata%5B*%5D.mark_infos%5B*%5D.url%3Bdata%5B*%5D.author.follower_count%2Cbadge%5B*%5D.topics%3Bsettings.table_of_content.enabled%3B&limit={}&offset={}&sort_by=default&platform=desktop"
    print("\n抓取知乎问题号{}:".format(question_id))
    while True:
        url = base_url.format(num, offset + i*num)
        ## 准备https头部
        headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
        # ## 睡眠
        sleep(0.5)
        ## 开始访问
        res = requests.get(url, headers=headers)
        ## 处理json数据, 获取回答和题目
        contents_list = get_info(res)
        ## 存储数据到csv中
        dataset = pd.DataFrame(contents_list, columns=["title", "comment"])
        if i==0:
            dataset.to_csv("data.csv", mode="a", header=False, index=False, sep="\t")
        else:
            dataset.to_csv("data.csv", mode="a", header=False, index=False, sep="\t")
        ## 判断截至条件
        if len(contents_list) < num:
            if len(contents_list) != 0:
                print("  已抓取{}条评论!".format(offset + i*num + len(contents_list)))
            print("问题号:" + question_id +"抓取完成!")
            break
        i += 1
        print("  已抓取{}条评论!".format(offset + i*num))

In [ ]:
def to_more_grap(question_id):
    '''
    用于获取相关问题的问题id
    '''
    # 准备网站
    base_url = "https://www.zhihu.com/api/v4/questions/" + question_id + "/similar-questions?include=data%5B*%5D.answer_count%2Cauthor%2Cfollower_count&limit=5"
    
    ## 准备https头部
    headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"}
    # ## 睡眠
    sleep(0.5)
    ## 开始访问
    res = requests.get(base_url, headers=headers)
    ## 处理json数据, 获取回答和题目
    contents_list = get_info(res, get_content=False)
    return contents_list

In [ ]:
def set_grap(question_id, question_num=1):
    '''
    集成爬虫
    '''
    # 爬取网站
    grap_web(question_id)
    question_num -= 1

    question_id = [question_id]
    i = 1
    while question_num:
        temp_i = i
        while len(question_id) == i:
            question_id += to_more_grap(question_id[temp_i-1])
            question_id = sorted(set(question_id),key=question_id.index)
            temp_i -= 1
            if temp_i == -1:
                print("因问题一致, 故抓取结束!")
                return
        grap_web(question_id[i])
        i += 1
        question_num -= 1
    print("抓取结束!")

In [ ]:
def main():
    # 默认id :339505719, 48510028, 20399824, 365863994
    print("输入抓取知乎对应问题的号码:")
    question_id = input()
    print("输入成功!")
    set_grap(question_id, question_num=100)

if __name__=="__main__":
    main()

输入抓取知乎对应问题的号码:
输入成功!

抓取知乎问题号365863994:
  已抓取5条评论!
  已抓取10条评论!
  已抓取15条评论!
  已抓取20条评论!
  已抓取25条评论!
  已抓取26条评论!
问题号:365863994抓取完成!

抓取知乎问题号264813121:
  已抓取5条评论!
  已抓取6条评论!
问题号:264813121抓取完成!

抓取知乎问题号386703419:
  已抓取5条评论!
  已抓取10条评论!
  已抓取15条评论!
  已抓取20条评论!
  已抓取25条评论!
  已抓取30条评论!
  已抓取35条评论!
  已抓取40条评论!
  已抓取45条评论!
  已抓取50条评论!
  已抓取55条评论!
  已抓取60条评论!
  已抓取65条评论!
  已抓取69条评论!
问题号:386703419抓取完成!

抓取知乎问题号20066579:
  已抓取5条评论!
  已抓取10条评论!
  已抓取15条评论!
  已抓取20条评论!
  已抓取25条评论!
  已抓取30条评论!
  已抓取35条评论!
问题号:20066579抓取完成!

抓取知乎问题号268092007:
  已抓取5条评论!
  已抓取6条评论!
问题号:268092007抓取完成!

抓取知乎问题号26522812:
  已抓取5条评论!
  已抓取10条评论!
  已抓取15条评论!
  已抓取20条评论!
  已抓取23条评论!
问题号:26522812抓取完成!

抓取知乎问题号36948840:
  已抓取5条评论!
  已抓取10条评论!
  已抓取15条评论!
  已抓取20条评论!
  已抓取21条评论!
问题号:36948840抓取完成!

抓取知乎问题号268641668:
  已抓取3条评论!
问题号:268641668抓取完成!

抓取知乎问题号21309575:
  已抓取5条评论!
  已抓取10条评论!
问题号:21309575抓取完成!

抓取知乎问题号62329311:
  已抓取5条评论!
  已抓取10条评论!
  已抓取15条评论!
  已抓取20条评论!
  已抓取25条评论!
  已抓取26条评论!
问题号:62329311抓取完成!

抓取知乎问题号32019629:
  已

**观察数据**

In [ ]:
import pandas as pd
dataset = pd.read_csv("../数据集/特朗普选举相关问题.csv", sep="\t", names=["title", "content"])

In [ ]:
dataset.head()

,title,content
0,参加「知乎母婴创作者盐沙龙」是一种怎样的体验？,一年前，我在知乎母婴赛道上还很孤独。我前面的大咖们，粉丝数是我的两倍，也产出了各种令人望其项...
1,参加「知乎母婴创作者盐沙龙」是一种怎样的体验？,体验就是：人生的奇遇，大抵如此。八个月前第一次在知乎写下回答，就此于母婴领域开启了笔耕不辍的...
2,参加「知乎母婴创作者盐沙龙」是一种怎样的体验？,现场速递！多图预警！很惭愧今年是替其他知友领奖，现场听到了大神@Cecilia和@翘囤奶爸的...
3,参加「知乎母婴创作者盐沙龙」是一种怎样的体验？,C位出道！现场版直播版，一大波大咖美图即将扑面而来如此高大上，有温度的沙龙，我是第一次参加。...
4,参加「知乎母婴创作者盐沙龙」是一种怎样的体验？,从观察者和分析师的角度，聊聊从「知乎母婴创作者盐沙龙」的表象下，看到的那些闪闪发光的价值点。...


In [ ]:
dataset["title"].value_counts()

2020 年美国大选投票计票开始，拜登 264:214 领先，离胜选仅差1州，对此你怎么看？         3549
美国媒体测算，拜登已获得超过270张选举人票，特朗普还有可能翻盘吗？                     2605
特朗普拒绝承认大选结果，称选举远没结束，后续美国局势会怎样发展？                        929
美媒测算大选全美票数已确定归属，拜登 306 : 232 特朗普，对此你怎么看？                623
特朗普女助理霍普·希克斯新冠检测阳性，曾多次陪同参加总统辩论，特朗普称已开始自我隔离，会影响到大选吗？     619
如何评价 2020 美国大选终场辩论上，总统候选人特朗普和拜登的表现？                     374
如何看待特朗普承认可能输掉20年大选，采访称“拜登将成为你们的总统”？                     284
特朗普新冠检测阳性之后，美国民主党总统候选人拜登新冠检测的结果为阴性，会对各方产生什么影响？          194
如何看待《纽约邮报》一则关于拜登的负面报道，于近日被Facebook和推特封杀？                193
特朗普为什么想当总统？                                             186
特朗普和拜登支持者在各地投票站外发生冲突，预计此类冲突会持续多久？后续形势将如何发展？             131
如何看待美国佐治亚州宣布将重新手工计算大选选票？美国总统大选结果有可能生变吗？                 109
特朗普当选了美国总统，希拉里接下来会怎样？                                   104
如何看待小特朗普称民主党总统候选人为「北京拜登」？                               103
特朗普已经确诊新冠，拜登是否也要隔离？后续二人会在隔离病房展开辩论吗？                      80
特朗普会因为新冠阳性而支持率得到提升吗？这时候拜登应该怎么做才合适？                       69
关键「摇摆州」威斯康星选情反转，拜登暂时反超特朗普，后续走向会如何？      

**目标:基于互动式, 使爬虫有目的式爬取(有点偏离主线功能, 暂时放弃)**

E4BDA0E5A5BD
63bd7491e5ed9446dd4ece995c6259eb
132577280007726624680888226644662008888
251503099356605

%E7%8B%BC%E6%AE%BF%E4%B8%8B
bf3ea761c1804667430746a40f87fff9

https://api.zhihu.com/search_v3?advert_count=0&correction=1&lc_idx=0&limit=5&offset=0&q=%E4%BD%A0%E5%A5%BD&search_hash_id=63bd7491e5ed9446dd4ece995c6259eb&show_all_topics=0&t=general&vertical_info=0%2C0%2C0%2C0%2C0%2C0%2C0%2C0%2C0%2C1

https://api.zhihu.com/search_v3?advert_count=0&correction=1&lc_idx=0&limit=20&offset=21&q=%E7%8B%BC%E6%AE%BF%E4%B8%8B&search_hash_id=bf3ea761c1804667430746a40f87fff9&show_all_topics=0&t=general&vertical_info=1%2C1%2C0%2C0%2C0%2C0%2C0%2C0%2C0%2C1

In [1]:
import requests

In [15]:
search_url = "https://www.zhihu.com/api/v4/search_v3?t=general&q=%E4%BD%A0%E5%A5%BD&correction=1&offset=0&limit=5&lc_idx=0&show_all_topics=0"
header = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63",
          "referer":"https://www.zhihu.com/search?q=%E4%BD%A0%E5%A5%BD&type=content",
         "accept": "*/*",
         "accept-encoding": "gzip, deflate, br",
         "sec-fetch-site": "same-origin"}

In [16]:
res = requests.get(search_url, headers=header)

In [17]:
res.text

'{\n    "error": {\n        "code": 10002,\n        "message": "10002:\\u8bf7\\u6c42\\u53c2\\u6570\\u5f02\\u5e38\\uff0c\\u8bf7\\u5347\\u7ea7\\u5ba2\\u6237\\u7aef\\u540e\\u91cd\\u8bd5"\n    }\n}\n'

# **导入elasticsearch库**

**连接elasticsearch库**

In [5]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
# import certifi
es = Elasticsearch(
    ['es-cn-09k1xdlqf000a7yyn.public.elasticsearch.aliyuncs.com'],
    http_auth=('elastic', 'Zy13720028637'),
    port=9200,
    use_ssl=False
)

{'title': 'One', 'tags': ['ruby']}


**创建索引**

In [ ]:
result = es.indices.create(index='my_index', ignore=400)
print(result)  # 成功->返回{'acknowledged': True, 'shards_acknowledged': True, 'index': 'news'}

**删除索引**

In [ ]:
result = es.indices.delete(index='my_index', ignore=[400, 404])
print(result)

**插入数据**
    
    1. index指文档所属索引名(可比作数据库名)
    2. doc_type指文档所属类型名(可比作数据库中的表, 但为了elastic search功能实现, 一个数据库就一个表)
    3. id指文档的主键绝对唯一
    4. body指文档内容, 字典类型

In [28]:
# 添加或更新数据,index，doc_type名称可以自定义，id可以根据需求赋值, body为内容
res = es.index(index="my_index", doc_type="my_type", id=1, body={"title": "One", "tags": ["ruby"]})  # 可忽略id号, 有index自动分配
print(res)

{'_index': 'my_index', '_type': 'my_type', '_id': '1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 8, '_primary_term': 1}


In [29]:
# 或者:ignore=409忽略文档已存在异常(不写ignore会抛出异常哦)
res = es.create(index="my_index",doc_type="my_type", id=1,ignore=409,body={"title": "Two", "tags": ["ruby", "geg"]})  # 不可忽略id号
print(res)

{'error': {'root_cause': [{'type': 'version_conflict_engine_exception', 'reason': '[my_type][1]: version conflict, document already exists (current version [2])', 'index_uuid': 'eDKh3MHEQ9Com-kKxeLlEQ', 'shard': '0', 'index': 'my_index'}], 'type': 'version_conflict_engine_exception', 'reason': '[my_type][1]: version conflict, document already exists (current version [2])', 'index_uuid': 'eDKh3MHEQ9Com-kKxeLlEQ', 'shard': '0', 'index': 'my_index'}, 'status': 409}


**更新数据**

In [34]:
data = {"title": "Three", "tags": ["ruby", "geg", "hah"], "data":"2020-11-12"}
# 老报错, 不知道为什么
result = es.update(index='my_index', doc_type='my_type', body=data, id=1, ignore=400)
print(result)

# 这个直接通过, 多好!!
result = es.update(index='my_index', doc_type='my_type', body=data, id=1, ignore=400)
print(result)

{'_index': 'my_index', '_type': 'my_type', '_id': '1', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 9, '_primary_term': 1}


**查询数据**

In [35]:
# 获取索引为my_index,文档类型为my_type的所有数据,result为一个字典类型
res = es.search(index="my_index", doc_type="my_type")
print(res)

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': 1, 'max_score': 1.0, 'hits': [{'_index': 'my_index', '_type': 'my_type', '_id': '1', '_score': 1.0, '_source': {'title': 'Three', 'tags': ['ruby', 'geg', 'hah'], 'data': '2020-11-12'}}]}}


In [36]:
# 或者这样写:搜索id=1的文档
result = es.get(index="my_index",doc_type="my_type",id=1)
print(result)

{'_index': 'my_index', '_type': 'my_type', '_id': '1', '_version': 3, '_seq_no': 9, '_primary_term': 1, 'found': True, '_source': {'title': 'Three', 'tags': ['ruby', 'geg', 'hah'], 'data': '2020-11-12'}}


**删除数据**

In [37]:
# 删除id=1的数据
result = es.delete(index="my_index",doc_type="my_type",id=1)
print(result)

{'_index': 'my_index', '_type': 'my_type', '_id': '1', '_version': 4, 'result': 'deleted', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 10, '_primary_term': 1}


**设置mapping 信息(搜索引擎的底层实现)**

    1. 创建新索引news, 并设置搜索引擎

In [39]:
mapping = {
    'properties': {
        'title': { # 属性
            'type': 'text',  # 指定类型text
            'analyzer': 'ik_max_word',  # 分词方法ik_max_word
            'search_analyzer': 'ik_max_word'  # 搜索分词方法ik_max_word
        }
    }
}
result = es.indices.create(index="news", ignore=400)
print(result)
result = es.indices.put_mapping(index='news', doc_type='politics', body=mapping)
print(result)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [news/yvTuawlPQKGZGRH_qjOmkA] already exists', 'index_uuid': 'yvTuawlPQKGZGRH_qjOmkA', 'index': 'news'}], 'type': 'resource_already_exists_exception', 'reason': 'index [news/yvTuawlPQKGZGRH_qjOmkA] already exists', 'index_uuid': 'yvTuawlPQKGZGRH_qjOmkA', 'index': 'news'}, 'status': 400}
{'acknowledged': True}


    2. 添加数据

In [40]:
datas = [
    {
        'title': '美国留给伊拉克的是个烂摊子吗',
        'url': 'http://view.news.qq.com/zt2011/usa_iraq/index.htm',
        'date': '2011-12-16'
    },
    {
        'title': '公安部：各地校车将享最高路权',
        'url': 'http://www.chinanews.com/gn/2011/12-16/3536077.shtml',
        'date': '2011-12-16'
    },
    {
        'title': '中韩渔警冲突调查：韩警平均每天扣1艘中国渔船',
        'url': 'https://news.qq.com/a/20111216/001044.htm',
        'date': '2011-12-17'
    },
    {
        'title': '中国驻洛杉矶领事馆遭亚裔男子枪击 嫌犯已自首',
        'url': 'http://news.ifeng.com/world/detail_2011_12/16/11372558_0.shtml',
        'date': '2011-12-18'
    }
]

for data in datas:
    es.index(index='news', doc_type='politics', body=data)

    3.1. 普通查询

In [42]:
import pprint
result = es.search(index='news', doc_type='politics')
pprint.pprint(result)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'CPYM8HUBHGd-2Bzvh5AB',
                    '_index': 'news',
                    '_score': 1.0,
                    '_source': {'date': '2011-12-16',
                                'title': '美国留给伊拉克的是个烂摊子吗',
                                'url': 'http://view.news.qq.com/zt2011/usa_iraq/index.htm'},
                    '_type': 'politics'},
                   {'_id': 'CfYM8HUBHGd-2Bzvh5A_',
                    '_index': 'news',
                    '_score': 1.0,
                    '_source': {'date': '2011-12-16',
                                'title': '公安部：各地校车将享最高路权',
                                'url': 'http://www.chinanews.com/gn/2011/12-16/3536077.shtml'},
                    '_type': 'politics'},
                   {'_id': 'CvYM8HUBHGd-2Bzvh5Br',
                    '_index': 'news',
                    '_score': 1.0,
                    '_source': {'date': '2011-12-17',
     

    3.2. 基于全文检索查询

In [44]:
dsl = {
    'query': {
        'match': {
            'title': '中国 领事馆'
        }
    }
}

result = es.search(index='news', doc_type='politics', body=dsl)
pprint.pprint(result)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'C_YM8HUBHGd-2Bzvh5CE',
                    '_index': 'news',
                    '_score': 3.992321,
                    '_source': {'date': '2011-12-18',
                                'title': '中国驻洛杉矶领事馆遭亚裔男子枪击 嫌犯已自首',
                                'url': 'http://news.ifeng.com/world/detail_2011_12/16/11372558_0.shtml'},
                    '_type': 'politics'},
                   {'_id': 'CvYM8HUBHGd-2Bzvh5Br',
                    '_index': 'news',
                    '_score': 0.642793,
                    '_source': {'date': '2011-12-17',
                                'title': '中韩渔警冲突调查：韩警平均每天扣1艘中国渔船',
                                'url': 'https://news.qq.com/a/20111216/001044.htm'},
                    '_type': 'politics'}],
          'max_score': 3.992321,
          'total': 2},
 'timed_out': False,
 'took': 3}


    4. 删除索引

In [45]:
res = es.indices.delete(index="news")
print(res)

{'acknowledged': True}
